In [1]:
import pandas as pd
from UtilsCreateDataFrame import *
import numpy as np
from dataclasses import dataclass, asdict
import matplotlib.pyplot as plt
import numpy_ext as npe
from statsmodels.tsa.stattools import adfuller
from Strategy import *
from Regression import *
import datetime


In [2]:
def loadCreatedDataFrame(path):
    df_ = pd.read_csv(path)
    df_ = df_.set_index(pd.to_datetime(df_.time))
    df_.drop(columns = ["time"], inplace = True)
    return df_
    

path = "BTC_ETH_2021-01.csv.gz"
df_ = loadCreatedDataFrame(path)


In [3]:
"""

with 1minute frequency a day has 1440 observations
let's try this on two week 

"""
index_input = df_.index
x, y = df_.to_numpy()[:,0], df_.to_numpy()[:,1]
window = int(1440)
last = 1440*7

rolling = npe.rolling_apply(linearRegression_np, window, x[:last], y[:last], n_jobs = 4)
df_beta = ResultDataFrame(rolling, index_input[:last])
df_beta


,beta,intercept,r2,res_std,res_mean,stationarity_pvalue,date_est
time,,,,,,,
2021-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:00:00+00:00
2021-01-01 00:01:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:01:00+00:00
2021-01-01 00:02:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:02:00+00:00
2021-01-01 00:03:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:03:00+00:00
2021-01-01 00:04:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:04:00+00:00
...,...,...,...,...,...,...,...
2021-01-07 23:55:00+00:00,0.023285,332.990880,0.826404,10.520993,-1.173185e-13,0.117492,2021-01-07 23:55:00+00:00
2021-01-07 23:56:00+00:00,0.023275,333.345485,0.826030,10.532041,6.805420e-14,0.112567,2021-01-07 23:56:00+00:00
2021-01-07 23:57:00+00:00,0.023264,333.709493,0.825598,10.544829,3.615873e-14,0.093566,2021-01-07 23:57:00+00:00


In [4]:
"""
TEST 1
trading_window = 1 day
"""

result_strategy = applyStrategyRolling(
    df_beta = df_beta,
    df_price = df_,
    trading_window_day = 1, 
    thr_pval = 0.1
)

df_trades = strategyDataFrame(result_strategy)
print("\nTotal Return: ", df_trades.ret_.prod(), "\n")
df_trades


Total Return:  1.2392849230595413 



,ret_,ret_type,enter_,exit_,beta,intercept,r2,res_std,res_mean,stationarity_pvalue,date_est
0,0.901889,Short,2021-01-02 12:46:00+00:00,2021-01-03 11:45:00+00:00,0.009511,451.716151,0.176213,4.590768,6.900159e-14,0.094863,2021-01-02 11:44:00+00:00
1,1.042345,Short,2021-01-04 03:59:00+00:00,2021-01-04 08:43:00+00:00,-0.064648,3057.470255,0.254715,63.921628,-7.124375e-13,0.096258,2021-01-04 03:44:00+00:00
2,1.060353,Long,2021-01-04 09:41:00+00:00,2021-01-04 14:01:00+00:00,-0.064648,3057.470255,0.254715,63.921628,-7.124375e-13,0.096258,2021-01-04 03:44:00+00:00
3,1.039959,Short,2021-01-05 00:15:00+00:00,2021-01-05 03:45:00+00:00,-0.064648,3057.470255,0.254715,63.921628,-7.124375e-13,0.096258,2021-01-04 03:44:00+00:00
4,0.991515,Long,2021-01-05 19:26:00+00:00,2021-01-06 03:47:00+00:00,0.039468,-223.496268,0.646754,26.455524,9.221266e-14,0.013973,2021-01-05 03:46:00+00:00
5,1.057345,Long,2021-01-06 03:50:00+00:00,2021-01-06 04:31:00+00:00,0.025292,232.794975,0.903117,9.882658,3.836931e-14,0.012305,2021-01-06 03:48:00+00:00
6,1.066384,Long,2021-01-06 05:45:00+00:00,2021-01-06 07:14:00+00:00,0.025292,232.794975,0.903117,9.882658,3.836931e-14,0.012305,2021-01-06 03:48:00+00:00
7,0.979015,Short,2021-01-06 11:09:00+00:00,2021-01-07 03:49:00+00:00,0.025292,232.794975,0.903117,9.882658,3.836931e-14,0.012305,2021-01-06 03:48:00+00:00
8,1.112923,Long,2021-01-07 18:49:00+00:00,2021-01-07 21:12:00+00:00,0.021754,396.990094,0.700663,16.099345,-1.157395e-13,0.099404,2021-01-07 09:51:00+00:00
9,0.981422,Long,2021-01-07 23:27:00+00:00,2021-01-08 09:52:00+00:00,0.021754,396.990094,0.700663,16.099345,-1.157395e-13,0.099404,2021-01-07 09:51:00+00:00


In [5]:
"""
TEST 2
trading window = 0.5 day 
"""

result_strategy = applyStrategyRolling(
    df_beta = df_beta,
    df_price = df_,
    trading_window_day = 0.5, 
    thr_pval = 0.1
)

df_trades = strategyDataFrame(result_strategy)
print("\nTotal Return: ", df_trades.ret_.prod(), "\n")
df_trades


Total Return:  1.2947522782969498 



,ret_,ret_type,enter_,exit_,beta,intercept,r2,res_std,res_mean,stationarity_pvalue,date_est
0,0.974749,Short,2021-01-02 12:46:00+00:00,2021-01-02 23:45:00+00:00,0.009511,451.716151,0.176213,4.590768,6.900159e-14,0.094863,2021-01-02 11:44:00+00:00
1,1.030641,Long,2021-01-03 04:38:00+00:00,2021-01-03 06:19:00+00:00,0.015768,265.239034,0.964695,3.973742,1.221344e-13,0.020711,2021-01-02 23:46:00+00:00
2,1.029433,Long,2021-01-03 07:34:00+00:00,2021-01-03 08:36:00+00:00,0.015768,265.239034,0.964695,3.973742,1.221344e-13,0.020711,2021-01-02 23:46:00+00:00
3,0.936762,Short,2021-01-03 08:57:00+00:00,2021-01-03 11:47:00+00:00,0.015768,265.239034,0.964695,3.973742,1.221344e-13,0.020711,2021-01-02 23:46:00+00:00
4,1.042345,Short,2021-01-04 03:59:00+00:00,2021-01-04 08:43:00+00:00,-0.064648,3057.470255,0.254715,63.921628,-7.124375e-13,0.096258,2021-01-04 03:44:00+00:00
5,1.060353,Long,2021-01-04 09:41:00+00:00,2021-01-04 14:01:00+00:00,-0.064648,3057.470255,0.254715,63.921628,-7.124375e-13,0.096258,2021-01-04 03:44:00+00:00
6,1.090030,Long,2021-01-05 14:53:00+00:00,2021-01-06 02:27:00+00:00,0.051429,-588.752607,0.874337,10.156933,2.960595e-13,0.002545,2021-01-05 14:26:00+00:00
7,1.121865,Long,2021-01-06 03:24:00+00:00,2021-01-06 08:03:00+00:00,0.026292,202.378827,0.889592,10.587005,-1.431349e-13,0.001090,2021-01-06 02:28:00+00:00
8,0.933953,Short,2021-01-06 11:09:00+00:00,2021-01-06 14:29:00+00:00,0.026292,202.378827,0.889592,10.587005,-1.431349e-13,0.001090,2021-01-06 02:28:00+00:00
9,1.112923,Long,2021-01-07 18:49:00+00:00,2021-01-07 21:12:00+00:00,0.021754,396.990094,0.700663,16.099345,-1.157395e-13,0.099404,2021-01-07 09:51:00+00:00


In [6]:
"""
TEST 2
trading window = 0.25 day 
"""

result_strategy = applyStrategyRolling(
    df_beta = df_beta,
    df_price = df_,
    trading_window_day = 0.25, 
    thr_pval = 0.1
)

df_trades = strategyDataFrame(result_strategy)
print("\nTotal Return: ", df_trades.ret_.prod(), "\n")
df_trades


Total Return:  1.1117833509203965 



,ret_,ret_type,enter_,exit_,beta,intercept,r2,res_std,res_mean,stationarity_pvalue,date_est
0,0.984020,Short,2021-01-02 12:46:00+00:00,2021-01-02 17:45:00+00:00,0.009511,451.716151,0.176213,4.590768,6.900159e-14,0.094863,2021-01-02 11:44:00+00:00
1,1.016332,Short,2021-01-02 21:01:00+00:00,2021-01-02 22:30:00+00:00,0.015730,266.977930,0.944086,3.770719,6.939634e-14,0.007821,2021-01-02 17:46:00+00:00
2,0.983294,Long,2021-01-03 04:38:00+00:00,2021-01-03 05:49:00+00:00,0.015776,265.018844,0.964644,3.978240,5.684342e-14,0.017149,2021-01-02 23:48:00+00:00
3,1.038634,Long,2021-01-03 06:04:00+00:00,2021-01-03 06:15:00+00:00,0.014676,298.476689,0.945411,4.556694,6.892265e-14,0.091637,2021-01-03 05:50:00+00:00
4,0.939674,Short,2021-01-03 08:42:00+00:00,2021-01-03 11:51:00+00:00,0.014676,298.476689,0.945411,4.556694,6.892265e-14,0.091637,2021-01-03 05:50:00+00:00
5,1.042345,Short,2021-01-04 03:59:00+00:00,2021-01-04 08:43:00+00:00,-0.064648,3057.470255,0.254715,63.921628,-7.124375e-13,0.096258,2021-01-04 03:44:00+00:00
6,1.013599,Long,2021-01-04 09:41:00+00:00,2021-01-04 09:45:00+00:00,-0.064648,3057.470255,0.254715,63.921628,-7.124375e-13,0.096258,2021-01-04 03:44:00+00:00
7,1.071711,Long,2021-01-05 14:53:00+00:00,2021-01-05 20:29:00+00:00,0.051447,-589.297534,0.874967,10.130693,3.742192e-14,0.002550,2021-01-05 14:28:00+00:00
8,1.129162,Long,2021-01-05 22:14:00+00:00,2021-01-06 02:32:00+00:00,0.035041,-72.457129,0.784610,14.442220,-8.842310e-15,0.092168,2021-01-05 20:31:00+00:00
9,1.114861,Long,2021-01-06 03:25:00+00:00,2021-01-06 08:03:00+00:00,0.026220,204.544642,0.891471,10.490482,1.070551e-13,0.002956,2021-01-06 02:33:00+00:00


In [7]:
from statsmodels.tsa.stattools import adfuller, kpss
x = np.random.standard_normal(1000)

In [8]:
1-kpss(x, regression = "ct")[1]

C:\Users\giova\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\giova\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(


0.9